### In this notebook California Census Data will be trained to predict the income class a person.

** Read in the census_data.csv data with pandas**

In [ ]:
import pandas as pd

In [ ]:
census = pd.read_csv("../input/census_data.csv")

In [ ]:
census.head()


** converting income_bracket to 0/1.**

In [ ]:
census['income_bracket'].unique()

In [ ]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [ ]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [ ]:
census.columns

** Import Tensorflow **

In [ ]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [ ]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [ ]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [ ]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

###  Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [ ]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

#### Create your model with tf.estimator



In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

** Train your model on the data, for at least 5000 steps. **

In [ ]:
model.train(input_fn=input_func,steps=5000)

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

** Use model.predict() and pass to input function. **

In [ ]:
predictions = list(model.predict(input_fn=pred_fn))

** Each item in your list will look like this: **

In [ ]:
predictions[0]

In [ ]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
final_preds[:10]

** print classification_report from sklearn.metrics  **

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,final_preds))